# Importações

In [1]:
import isodate
import pandas as pd

## Leitura dos dados

Anteriormente, tentamos ler o arquivo completo disponível no kaggle. Entretando, com a limitação de memória RAM que temos, extraímos 300.000 linhas do dataset.

In [2]:
df = pd.read_csv('../data/arquivo.csv')

In [3]:
df.shape

(299999, 27)

## Verificar valores nulos ou vazios

Verificamos que a coluna totalTravelDistance possui 16.814 valores não preenchidos, sendo uma pequena parte dos dados. Com isso, vamos análisar a necessidade deles e se podem ser removidos.

Para as outras duas colunas, notamos que são informações possivelmente, irrelevantes para a predição.

In [4]:
df.isna().sum()

legId                                    0
searchDate                               0
flightDate                               0
startingAirport                          0
destinationAirport                       0
fareBasisCode                            0
travelDuration                           0
elapsedDays                              0
isBasicEconomy                           0
isRefundable                             0
isNonStop                                0
baseFare                                 0
totalFare                                0
seatsRemaining                           0
totalTravelDistance                  16814
segmentsDepartureTimeEpochSeconds        0
segmentsDepartureTimeRaw                 0
segmentsArrivalTimeEpochSeconds          0
segmentsArrivalTimeRaw                   0
segmentsArrivalAirportCode               0
segmentsDepartureAirportCode             0
segmentsAirlineName                      0
segmentsAirlineCode                      0
segmentsEqu

In [5]:
df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H29M,0,False,False,...,1650223560,2022-04-17T15:26:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,8940,947,coach
1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,False,False,...,1650200400,2022-04-17T09:00:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9000,947,coach
2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,False,False,...,1650218700,2022-04-17T14:05:00.000-04:00,BOS,ATL,Delta,DL,Boeing 757-200,9000,947,coach
3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H32M,0,False,False,...,1650227460,2022-04-17T16:31:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9120,947,coach
4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H34M,0,False,False,...,1650213180,2022-04-17T12:33:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9240,947,coach


Abaixo, vemos que o tipo dos dados estão corretos.

In [6]:
df.dtypes

legId                                 object
searchDate                            object
flightDate                            object
startingAirport                       object
destinationAirport                    object
fareBasisCode                         object
travelDuration                        object
elapsedDays                            int64
isBasicEconomy                          bool
isRefundable                            bool
isNonStop                               bool
baseFare                             float64
totalFare                            float64
seatsRemaining                         int64
totalTravelDistance                  float64
segmentsDepartureTimeEpochSeconds     object
segmentsDepartureTimeRaw              object
segmentsArrivalTimeEpochSeconds       object
segmentsArrivalTimeRaw                object
segmentsArrivalAirportCode            object
segmentsDepartureAirportCode          object
segmentsAirlineName                   object
segmentsAi

Ao tentar identificar os dias corridos da busca para o dia do voo, nos deparamos que a coluna estava em formato str, sendo necessária a conversão para datetime.

In [7]:
df['flightDate'] = pd.to_datetime(df['flightDate'])
df['searchDate'] = pd.to_datetime(df['searchDate'])

## Alteração de valores

Quando iniciamos o treinamento do modelo, identificamos que as colunas de data precisam estar como numéricas, então, as convertemos para dias.

Para essa transformação, dividimos os nanossegundos e dividimos por 864.000 segundos (1 dia = 864.000 segundos).

In [8]:
df['searchDateInDays'] = df['searchDate'].astype(int) // 86400
df['flightDateInDays'] = df['flightDate'].astype(int) // 86400

Incluimos uma coluna para identificar o tempo do dia da busca e o dia do voo.

In [9]:
df['daysDifferenceBetweenSearchAndFlightDate'] = (df['flightDate'] - df['searchDate']).dt.days
df['daysDifferenceBetweenSearchAndFlightDate']

0          1
1          1
2          1
3          1
4          1
          ..
299994    24
299995    24
299996    24
299997    24
299998    24
Name: daysDifferenceBetweenSearchAndFlightDate, Length: 299999, dtype: int64

In [10]:
df['travelDuration'].unique()

array(['PT2H29M', 'PT2H30M', 'PT2H32M', ..., 'PT21H57M', 'PT20H20M',
       'PT20H44M'], shape=(1203,), dtype=object)

Identificamos que a coluna travelDuration, está no formato ISO 8601 e para convertermos em segundos, utilizamos a lib isodate, que consegue identificar esse formato, converter e retornamos o total de segundos.

In [11]:
df['travelDurationInSeconds'] = df['travelDuration'].apply(lambda x: isodate.parse_duration(x).total_seconds())
df['travelDurationInSeconds']


0          8940.0
1          9000.0
2          9000.0
3          9120.0
4          9240.0
           ...   
299994    44820.0
299995    20640.0
299996    28980.0
299997    36000.0
299998    26880.0
Name: travelDurationInSeconds, Length: 299999, dtype: float64

Também, separamos as taxas e impostos do valor da passagem.

In [12]:
df['taxesFare'] = df['totalFare'] - df['baseFare']
df['taxesFare']

0         30.93
1         30.93
2         30.93
3         30.93
4         30.93
          ...  
299994    40.57
299995    35.39
299996    35.39
299997    42.67
299998    38.11
Name: taxesFare, Length: 299999, dtype: float64

Notamos que o nome das companhias aéreas, estão na mesma coluna, separadas por '||'. Então decidimos separar os trecho em colunas.

In [13]:
df['segmentsAirlineName'].unique()

array(['Delta', 'JetBlue Airways', 'American Airlines||American Airlines',
       'United||United', 'Spirit Airlines||Spirit Airlines',
       'American Airlines', 'JetBlue Airways||JetBlue Airways',
       'Frontier Airlines', 'United',
       'Frontier Airlines||Frontier Airlines', 'Spirit Airlines',
       'Delta||Delta', 'Delta||United',
       'Delta||Cape Air||Cape Air||Delta', 'Delta||Delta||Delta',
       'Delta||Cape Air||Cape Air||United', 'United||Delta',
       'United||United||Delta',
       'Frontier Airlines||Frontier Airlines||Frontier Airlines',
       'Alaska Airlines',
       'Spirit Airlines||Spirit Airlines||Spirit Airlines',
       'Delta||United||Delta', 'United||United||Alaska Airlines',
       'Alaska Airlines||Alaska Airlines',
       'Boutique Air||Boutique Air||American Airlines',
       'Sun Country Airlines||Sun Country Airlines',
       'Cape Air||Cape Air||Delta', 'Cape Air||Cape Air',
       'Alaska Airlines||Delta', 'Delta||Alaska Airlines',
       'De

In [14]:
def get_segment(x, index):
    try:
        if (not isinstance(x, str)): 
            return None
        
        return x.split('||')[index]
    
    except IndexError:
        return None
    

In [15]:
df['segment1AirlineName'] = df['segmentsAirlineName'].apply(lambda x: get_segment(x, 0))
df['segment2AirlineName'] = df['segmentsAirlineName'].apply(lambda x: get_segment(x, 1))
df['segment3AirlineName'] = df['segmentsAirlineName'].apply(lambda x: get_segment(x, 2))
df['segment4AirlineName'] = df['segmentsAirlineName'].apply(lambda x: get_segment(x, 3))

df[['segment1AirlineName', 'segment2AirlineName', 'segment3AirlineName', 'segment4AirlineName']]

,segment1AirlineName,segment2AirlineName,segment3AirlineName,segment4AirlineName
0,Delta,None,None,None
1,Delta,None,None,None
2,Delta,None,None,None
3,Delta,None,None,None
4,Delta,None,None,None
...,...,...,...,...
299994,United,United,None,None
299995,United,United,None,None
299996,United,United,None,None
299997,Delta,Delta,None,None


Fizemos o mesmo para o tipo de cabine.

In [16]:
df['segmentsCabinCode'].unique()

array(['coach', 'coach||coach', 'coach||coach||coach||coach',
       'coach||coach||coach', 'first', 'premium coach', 'first||coach',
       'business', 'premium coach||premium coach', 'first||first',
       'coach||premium coach', 'coach||first', 'first||coach||coach',
       'business||first||first', 'business||coach||coach',
       'business||coach', 'business||business||coach',
       'coach||business||business', 'business||coach||business',
       'premium coach||premium coach||coach', 'coach||business',
       'coach||coach||business', 'business||business',
       'coach||coach||first||coach', 'premium coach||coach',
       'first||first||first', 'coach||first||first',
       'coach||coach||first', 'coach||first||coach',
       'coach||business||coach', 'coach||coach||premium coach',
       'premium coach||coach||coach', 'first||first||coach'], dtype=object)

In [17]:
df['segment1CabinCode'] = df['segmentsCabinCode'].apply(lambda x: get_segment(x, 0))
df['segment2CabinCode'] = df['segmentsCabinCode'].apply(lambda x: get_segment(x, 1))
df['segment3CabinCode'] = df['segmentsCabinCode'].apply(lambda x: get_segment(x, 2))
df['segment4CabinCode'] = df['segmentsCabinCode'].apply(lambda x: get_segment(x, 3))

df[['segment1CabinCode', 'segment2CabinCode', 'segment3CabinCode', 'segment4CabinCode']]

,segment1CabinCode,segment2CabinCode,segment3CabinCode,segment4CabinCode
0,coach,None,None,None
1,coach,None,None,None
2,coach,None,None,None
3,coach,None,None,None
4,coach,None,None,None
...,...,...,...,...
299994,coach,coach,None,None
299995,coach,coach,None,None
299996,coach,coach,None,None
299997,coach,coach,None,None


Fizemos o mesmo para segmentsEquipmentDescription. Embora não conseguimos visualizar todos os dados, pressupomos, que essa coluna também segue 4 segmentos, conforme as anteriores.

In [18]:
df['segmentsEquipmentDescription'].unique()

array(['Airbus A321', 'Boeing 757-200', nan, ...,
       'Airbus A220-100||Airbus A321||Boeing 757-300',
       'Embraer 175||Boeing 737-900||Boeing 757-300',
       'Airbus A320||Airbus A319||Airbus A220-100'],
      shape=(2763,), dtype=object)

In [19]:
df['segment1EquipmentDescription'] = df['segmentsEquipmentDescription'].apply(lambda x: get_segment(x, 0))
df['segment2EquipmentDescription'] = df['segmentsEquipmentDescription'].apply(lambda x: get_segment(x, 1))
df['segment3EquipmentDescription'] = df['segmentsEquipmentDescription'].apply(lambda x: get_segment(x, 2))
df['segment4EquipmentDescription'] = df['segmentsEquipmentDescription'].apply(lambda x: get_segment(x, 3))

df[['segment1EquipmentDescription', 'segment2EquipmentDescription', 'segment3EquipmentDescription', 'segment4EquipmentDescription']]

,segment1EquipmentDescription,segment2EquipmentDescription,segment3EquipmentDescription,segment4EquipmentDescription
0,Airbus A321,None,None,None
1,Airbus A321,None,None,None
2,Boeing 757-200,None,None,None
3,Airbus A321,None,None,None
4,Airbus A321,None,None,None
...,...,...,...,...
299994,Boeing 737-800,Airbus A320,None,None
299995,Embraer 175 (Enhanced Winglets),Boeing 737-800,None,None
299996,Embraer 175 (Enhanced Winglets),Boeing 737 MAX 9,None,None
299997,Airbus A220-100,Embraer 175,None,None


In [20]:
df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segment3AirlineName,segment4AirlineName,segment1CabinCode,segment2CabinCode,segment3CabinCode,segment4CabinCode,segment1EquipmentDescription,segment2EquipmentDescription,segment3EquipmentDescription,segment4EquipmentDescription
0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H29M,0,False,False,...,None,None,coach,None,None,None,Airbus A321,None,None,None
1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,False,False,...,None,None,coach,None,None,None,Airbus A321,None,None,None
2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,False,False,...,None,None,coach,None,None,None,Boeing 757-200,None,None,None
3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H32M,0,False,False,...,None,None,coach,None,None,None,Airbus A321,None,None,None
4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H34M,0,False,False,...,None,None,coach,None,None,None,Airbus A321,None,None,None


Após as alterações identificamos que a duração do voo em segundos já estava presente, sendo representada pela coluna segmentsDurationInSeconds. Também, notamos que as demais colunas estão corretas, não precisando ser modificadas.

## Remoção de dados duplicados

In [21]:
df[df.duplicated()]

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segment3AirlineName,segment4AirlineName,segment1CabinCode,segment2CabinCode,segment3CabinCode,segment4CabinCode,segment1EquipmentDescription,segment2EquipmentDescription,segment3EquipmentDescription,segment4EquipmentDescription


Identificamos que não há dados duplicados.

## Salvar dados

In [22]:
df.to_csv('../data/arquivo_preprocessado.csv', index=False)